<a href="https://colab.research.google.com/github/Elbereth-Elentari/Book_recommender/blob/master/baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [31]:
cat = pd.read_json('/content/drive/My Drive/Library_catalogue_preprocessed.jl', lines=True, orient='records').fillna('')
cat = cat[cat['interesting'] != '']
cat.replace({'interesting':'yes'}, 1, inplace=True)
cat.replace({'interesting':'no'}, 0, inplace=True)
cat.replace({'language':'pl'}, 1, inplace=True)
cat.replace({'language':'en'}, 0, inplace=True)

def join_tokens(row):
    return ' '.join(row['tokens'])

cat['tokens'] = cat.apply(join_tokens, axis=1)
cat.head()

,title,author,storage,WD_signature,publisher,year,pages,language,tokens,interesting
9,Mit sztuk zen w kształtowaniu się kultury arty...,"Sokołowski, Michał Andrzej.",magazyn,BF576 .B37165 2020,Kirin,2020,654,1,mita sztuka zen kształtowaniu kultura artystyc...,0
160,Magistrala CAN od A do Z : diagnostyka i progr...,"Kardaś, Paweł.",magazyn,TJ223.M53 K37 2020,Wydawnictwo Atnel,2020,204,1,magistrala can diagnostyk programowanie język c,0
221,Japoński socjolekt młodzieżowy jako manifestac...,"Duc-Harada, Patrycja.",,,Wydawnictwo Uniwersytetu Jagiellońskiego,2020,358,1,japoński socjolekt młodzieżowy manifestacja św...,0
279,Czysty kod w Pythonie,"Kapil, Sunil.",magazyn,QA76.73.P98 K365165 2020,Helion SA,2020,176,1,czysty koda pythonie,1
295,Żony nazistów : kobiety kochające zbrodniarzy,"Wyllie, James.",,DD245 .W85165 2020,Wydawnictwo Agora,2020,359,1,żona nazista kobieta kochać zbrodniarz,0


In [90]:
X = cat[['year', 'pages', 'language']]
y = cat['interesting']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

scaler = MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [95]:
best_k = 0
best_acc = 0

for k in range(1,10):
    knn = KNeighborsClassifier(n_neighbors=k).fit(X_train_scaled, y_train)
    acc = accuracy_score(y_test, knn.predict(X_test_scaled))
    if acc > best_acc:
        best_acc = acc
        best_k = k

print(f'The highest accuracy is {best_acc}, for k == {k}')

The highest accuracy is 0.7587719298245614, for k == 9


In [106]:
logreg = LogisticRegression().fit(X_train_scaled, y_train)
accuracy_score(y_test, logreg.predict(X_test_scaled))

0.7258771929824561